In [186]:
# Main and sub-functions section
# TODO: 
#    1. use SDDI format instead of csv (check this out, https://pylhc.github.io/sdds/index.html)

import urllib
from bs4 import BeautifulSoup
import numpy as np

def get_group():
    # this function get Datagroup list
    
    # site and query info
    site = 'https://ops.aps.anl.gov/cgi-bin/oagMonitorDataReview.cgi'
    query = {'ListDataGroups': '1'}
    # constuct url
    url = f'{site}?{urllib.parse.urlencode(query)}'
    
    print(url)
    
    # get the page
    resp = urllib.request.urlopen(url)
    the_page = resp.read()
    resp.close()
    # parse the page
    # see this page "https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser" for parser selection
    soup= BeautifulSoup(the_page, 'lxml')
    # get Data_Group list
    group_list = soup.body.get_text().split()
    
    return group_list


def get_PVlist(datagroup):
    # this function get List of PVs under a specific DataGroup
    # DataGroup must be a list or tuple and the function only process the first element (for now)
    
    if not isinstance(datagroup,(list, tuple, str)):
        raise TypeError("Input must be a tuple or list or string")
    elif isinstance(datagroup,(list, tuple)) and len(datagroup)>1 :
        print('Input has more than one element, process only the first one!!')
        datagroup=datagroup[0]

    # site and query info
    site = 'https://ops.aps.anl.gov/cgi-bin/oagMonitorDataReview.cgi'
    query = {'DataGroup': datagroup,
             'ListReadbackNames': '1',}
    # constuct url
    url = f'{site}?{urllib.parse.urlencode(query)}'

    # get the page
    resp = urllib.request.urlopen(url)
    the_page = resp.read()
    resp.close()
    # parse the page
    # see this page "https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser" for parser selection
    soup= BeautifulSoup(the_page, 'lxml')
    # get Data_Group list
    PV_list = soup.body.get_text().split()

    if not PV_list:
        print(f'The requested DataGroup "{datagroup}" does not exist!!')
    else:
        return PV_list

def get_PVdata(datagroup, pvlist, date_range=None):

    if not isinstance(datagroup,(list, tuple, str)):
        raise TypeError("Input must be a tuple or list or string")
    elif isinstance(datagroup,(list, tuple)) and len(datagroup)>1 :
        print('Input more than one DataGroup, query only the first one!!')
        datagroup=datagroup[0]        

    PVlist=get_PVlist(datagroup)
    #print(PVlist)
        
    # site and query info
    site = 'https://ops.aps.anl.gov/cgi-bin/oagMonitorDataReview.cgi'
    query = {'DataGroup': datagroup,
             datagroup+'_ControlReadbackName': '18',
            'ExportCSV': 'Export+Data+(CSV)',
            'StartYear': '2022',
            'StartMonth':'10',
            'StartDay':  '11',
            'StartHour': '0',
            'EndYear':   '2022',
            'EndMonth':  '10',
            'EndDay':    '11',
            'EndHour':   '24',}
    # constuct url
    url = f'{site}?{urllib.parse.urlencode(query)}'
    # need this freaking line to parse address correctly!! maybe there is a way to do it in erlencode() but I cannot figure it out!!
    url = urllib.parse.unquote_plus(url)

    #print(url)
    # get the data (csv format)
    resp = urllib.request.urlopen(url)
    #print(resp.status)
    lines = [l.decode('utf-8') for l in resp.readlines()]
    resp.close()

    # header
    header = lines[2].rstrip().split(',')

    # read data
    arr=np.array([])
    for i in range(3,len(lines)):
        da_=lines[i].rstrip().split(',')
    if not da_[0]=='':   # skip empty line
        arr = np.append(arr, np.array(da_).astype('float'))

    # reshape array, now we have the data
    arr = np.reshape(arr,(-1,len(header)))    

    return header, arr



In [187]:
#### get the data
PVgroup=('1ID')
PVlist=('S:SRcurrentAI')

header, da = get_PVdata(PVgroup, PVlist)

In [188]:
#### Plot the result

## convert epoch time to human readable time
#import datetime

eptime = arr[:,1]

## plot result
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib qt5
#%matplotlib inline

#humantime = mdate.date2num(datetime.utcfromtimestamp(eptime)) 

fig,ax=plt.subplots()
for i in range(2, len(header)):
    ax.plot(eptime.astype('datetime64[s]'), arr[:,i],'o', label=header[i])

# set proper timezone to display correct time. 
ax.xaxis_date(tz='US/Central')

plt.xlabel('Time (sec)')
plt.ylabel('Temperature(degC)')
ax.legend()

plt.show()

In [391]:
from datetime import datetime
datagroup='1ID'
pvlist = ['S:SRcurrentAI','ID01DSEnergy']
PVlist=get_PVlist(datagroup)
start_time=5
end_time='now'
#####

index_list = []
for pv in pvlist:
    index_list.append(PVlist.index(pv))

if end_time=='now':
    endtime  = datetime.now()
    endyear  = endtime.year
    endmonth = endtime.month
    endday   = endtime.day
    endhour  = endtime.hour
    if start_time.isdigit():
        print(f'start_time is digit and is {start_time}')
    

# site and query info
site = 'https://ops.aps.anl.gov/cgi-bin/oagMonitorDataReview.cgi'

query = {'DataGroup': datagroup,
         datagroup+'_ControlReadbackName': index_list,
        'ExportCSV': 'Export+Data+(CSV)',
        'StartYear': '2022',
        'StartMonth':'10',
        'StartDay':  '11',
        'StartHour': '0',
        'EndYear':   endyear,
        'EndMonth':  endmonth,
        'EndDay':    endday,
        'EndHour':   endhour,}


# constuct url
url = f'{site}?{urllib.parse.urlencode(query,True)}'
# need this freaking line to parse address correctly("+" & "()" sign)!! maybe there is a way to do it in erlencode() but I cannot figure it out!!
url = urllib.parse.unquote_plus(url)
print(url)

resp = urllib.request.urlopen(url)
#print(resp.status)

# treat as a html page
the_page = resp.read()
soup= BeautifulSoup(the_page, 'lxml')
resp.close()

try:
    header = soup.title.get_text()
    if 'Error' in header:
        print(f'Respond: {header}')
        arr=None
    else:
        print(f'Undefined Respond: {header}')
        arr=None
except:
    # title is None, so likely a text file
    print('Data download successfully...')
    body = soup.body.get_text().rstrip().split()
    header = body[1].split(',')
    arr=np.array([])
    for i in range(2,len(body)):
        da_=body[i].rstrip().split(',')
        if not da_[0]=='':   # skip empty line
            arr = np.append(arr, np.array(da_).astype('float'))
            # reshape array, now we have the data
            arr = np.reshape(arr,(-1,len(header)))   


print(header)
#print(lines)


AttributeError: 'int' object has no attribute 'isdigit'

In [392]:
print(int(2.4))

2
